In [162]:
# %pip install qubovert
# %pip install qiskit
# %pip install pandas
# %pip install numpy
# %pip install matplotlib

In [163]:
from qubovert import boolean_var, QUBO
import pandas as pd
from qubovert.sim import anneal_qubo
import numpy as np
import re
from qiskit import QuantumCircuit
from qiskit.providers.basic_provider import BasicProvider
from qiskit.quantum_info import Statevector
from qiskit.circuit.quantumcircuit import QuantumCircuit as QCtype
from itertools import product
from qiskit import transpile
import numpy as np
import os
from qiskit.circuit.library import XGate, IGate, RZGate, SXGate, CXGate, U3Gate, ZGate, HGate, TGate, TdgGate, SGate, CCXGate, RXGate, Reset
# import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import itertools

### Creating the quantum circuit to optimize

Quantum circuit (qc) with 5 qubits
and apply cnot gates as in Fig.2 of the paper

In [164]:
qc2 = QuantumCircuit(5)
qc2.cx(0,2)
qc2.cx(1,3)
qc2.cx(0,1)
qc2.cx(3,4)
qc2.cx(1,3)
qc2.cx(0,3)
qc2.cx(1,4)
qc2.cx(1,3)
qc2.cx(2,4)

qc2.draw()

q_0: ──■─────────■─────────■─────────────────
       │       ┌─┴─┐       │                 
q_1: ──┼────■──┤ X ├──■────┼────■────■───────
     ┌─┴─┐  │  └───┘  │    │    │    │       
q_2: ┤ X ├──┼─────────┼────┼────┼────┼────■──
     └───┘┌─┴─┐     ┌─┴─┐┌─┴─┐  │  ┌─┴─┐  │  
q_3: ─────┤ X ├──■──┤ X ├┤ X ├──┼──┤ X ├──┼──
          └───┘┌─┴─┐└───┘└───┘┌─┴─┐└───┘┌─┴─┐
q_4: ──────────┤ X ├──────────┤ X ├─────┤ X ├
               └───┘          └───┘     └───┘

### Gate class

The `Gate` class is used to store information about
the gates such as the name of the gate, the qiskit
function that implements it and the qubits it operates on

In [165]:
class Gate:
    convertions = {
        'cx': CXGate
    }
    def __init__(self, name:str, qubits:list, params:list=None):
        self.name = name
        self.qubits = qubits
        self.params = params
        try:
            self.func = self.convertions[name]
        except KeyError:
            print("This gate is not supported")
    def __str__(self):
        s = f"(Gate: name: {self.name}, qubits: {self.qubits}"
        if self.params is not None:
            s += f", Params: {self.params}"

        s += ")"
        return s

    def __repr__(self):
        return self.__str__()


def get_qubits_idxs(qubits:list)->list[int]:
    '''Given a list of qubits, it returns a list of their indexes''' 
    idxs = []
    for q in qubits:
        idxs.append(q._index)

    return idxs


### Iterating over gates in a qiskit ciruit

We can iterate over the gates in a circuit in this way

The problem is that the informations are not easy to use (see below)

In [166]:
for i in qc2.data:
    print(i)

CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'q'), 0), Qubit(QuantumRegister(5, 'q'), 2)), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'q'), 1), Qubit(QuantumRegister(5, 'q'), 3)), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'q'), 0), Qubit(QuantumRegister(5, 'q'), 1)), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'q'), 3), Qubit(QuantumRegister(5, 'q'), 4)), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'q'), 1), Qubit(QuantumRegister(5, 'q'), 3)), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[])

So we can use the class we defined before and the 
`get_qubits_idxs()` function to make them nicer to
work with

In [167]:

def get_gates_from_circuit(qc) -> list[Gate]:
    '''Given a QuantumCircuit, it returns a list of Gates'''
    gate_list = []
    # Iterate over all the gates in the qc
    for i in qc.data:
        op = i.operation.name
        qubits = i.qubits
        # Convert the qubits to a list of indexes
        qubits_indexes = get_qubits_idxs(qubits)

        # Add to the end of the gate_list the Gate
        gate_list.append(Gate(op, qubits_indexes))
    return gate_list

gate_list = get_gates_from_circuit(qc2)
for g in gate_list:
    print(g)
    

(Gate: name: cx, qubits: [0, 2])
(Gate: name: cx, qubits: [1, 3])
(Gate: name: cx, qubits: [0, 1])
(Gate: name: cx, qubits: [3, 4])
(Gate: name: cx, qubits: [1, 3])
(Gate: name: cx, qubits: [0, 3])
(Gate: name: cx, qubits: [1, 4])
(Gate: name: cx, qubits: [1, 3])
(Gate: name: cx, qubits: [2, 4])


### Time slices

Time slices are basically a list of gates but it is convenient to make a class
that represents a time slice so that we can use them more easily (for example with the function
`any_in` we can see if any qubit in a list is already present in the time slice)

In [ ]:
# The TimeSlice class is a fancy list of gates
class TimeSlice:
    def __init__(self, idx, num_qubits=5):
        self.idx = idx
        self.num_qubits = num_qubits
        self.gates = []
        # involved_qubits stores the qubits already involved in the slice
        # it is useful while performing slicing
        self.involved_qubits = []
        self.graph = [] # graph of interaction of the qubits

    def append(self, gate:Gate):
        '''Adds the gate to the timeSlice making sure that the qubits are valid qubits'''
        # check if the gate we want to add to the slice
        # operates on the correct qubits
        for i in gate.qubits:
            if i >= self.num_qubits:
                raise(f"The TimeSlice has {self.num_qubits} but a gate acting on qubit {i} was added")
            if i in self.involved_qubits:
                raise(f"Error: qubit {i} is already present in TimeSlice{self.idx}")

            self.involved_qubits.append(i)
        
        # create the graph of the interactions of the qubits involved in the same gate
        for l in list(itertools.combinations(gate.qubits, 2)):
            self.graph.append(l)
            
        self.gates.append(gate)

    def contains_any(self, qubits: list):
        '''Returns true if at least one of the qubits is already used in the timeSlice'''
        return True if [i for i in qubits if i in self.involved_qubits] else False

    def __str__(self):
        s = f"(TimeSlice{self.idx}, gates: {self.gates}, graph: {self.graph})"

        return s

    def __repr__(self):
        return self.__str__()

### Slicing the circuit

We can now slice the circuit loosely following algorithm 1 of the paper

In [169]:

def slice_circuit(qc):
    '''Applies the ADDGATE funciton of the 
    paper (Algorithm 1) to the QuantumCircuit qc'''
    gate_list = get_gates_from_circuit(qc)
    # Create an empty list of slices
    slices:TimeSlice= []
    for g in gate_list:
        # "if gate in slices[t]" <- this check should not be needed

        # if this is the first slice we simply add the qubit
        if len(slices) == 0:
            slices.append(TimeSlice(len(slices)))
            slices[-1].append(g) # given a list l, l[-1] is the last element of the list

        # check if any qubit is already involved in the current time slice
        elif slices[-1].contains_any(g.qubits):
            # if it is we have to create a new slice and add the qubit to it
            slices.append(TimeSlice(len(slices)))
            slices[-1].append(g)
        else:
            slices[-1].append(g)

    return slices


In [170]:
slice_circuit = slice_circuit(qc2)
for i in slice_circuit:
    print(i.graph)

[(0, 2), (1, 3)]
[(0, 1), (3, 4)]
[(1, 3)]
[(0, 3), (1, 4)]
[(1, 3), (2, 4)]


In [171]:
stuff = [1, 2, 3]
for subset in itertools.combinations(stuff, 2):
    print(subset)

print(list(itertools.combinations(stuff, 2)))

(1, 2)
(1, 3)
(2, 3)
[(1, 2), (1, 3), (2, 3)]
